In [2]:
import numpy as np
import pandas as pd
import requests
from census import Census
from sqlalchemy import create_engine

api_key = "85ac64b6b5a9c0901b00329d1ef41f0c53ccfc98"

In [3]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
def project_data(api, year):
    
    c = Census(api_key, year=year)
    
    census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B17012_002E",
                          "B19301_001E",
                          "B17001_002E",
                          "B25064_001E",
                          "B23025_005E",
                          "B02001_003E",
                          "B03001_003E",
                          "B02001_002E",
                          "B17001A_002E",
                          "B17001I_002E",
                          "B17001B_002E"), {'for': 'state:*'})

    # Convert to DataFrame
    census_pd = pd.DataFrame(census_data)

    # Column Reordering
    census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "B19013_001E": "Median Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "B25064_001E": "Median Cost of Rent",
                                      
                                      "B02001_003E":"Black Population",
                                      "B03001_003E":"Hispanic Population",
                                      "B02001_002E":"White Population",
                                      "B17001A_002E":"White Poverty Count",
                                      "B17001I_002E":"Hispanic Poverty Count",
                                      "B17001B_002E":"Black Poverty Count",
                                      "NAME": "Name", "state": "State"})

    # Add in Poverty Rate (Poverty Count / Population)
    census_pd["Poverty Rate"] = 100 * \
        census_pd["Poverty Count"].astype(
            int) / census_pd["Population"].astype(int)

    # Add in Employment Rate (Employment Count / Population)
    census_pd["Unemployment Rate"] = 100 * \
        census_pd["Unemployment Count"].astype(
            int) / census_pd["Population"].astype(int)

    census_pd["Black Poverty Rate"] = 100 * census_pd["Black Poverty Count"].astype(
            int) / census_pd["Black Population"].astype(int)

    census_pd["White Poverty Rate"] = 100 * census_pd["White Poverty Count"].astype(
            int) / census_pd["White Population"].astype(int)

    census_pd["Hispanic Poverty Rate"] = 100 * census_pd["Hispanic Poverty Count"].astype(
            int) / census_pd["Hispanic Population"].astype(int)





    # Final DataFrame
    census_pd = census_pd[["State", "Name", "Population", "Median Household Income",
                       "Per Capita Income", "Poverty Rate", "Unemployment Rate",
                       "Median Cost of Rent", "Black Poverty Rate", "Hispanic Poverty Rate", "White Poverty Rate"]]

    return census_pd

In [4]:
data_2011 = project_data(api_key,2011)
data_2012 = project_data(api_key,2012)
data_2013 = project_data(api_key,2013)
data_2014 = project_data(api_key,2014)
data_2015 = project_data(api_key,2015)
data_2016 = project_data(api_key,2016)
data_2017 = project_data(api_key,2017)

In [106]:
# creating sql connection
connection_string = "postgres:Riceuniv19!@localhost:5432/project2"
engine = create_engine(f'postgresql://{connection_string}')

In [114]:
def to_sql(df_name, table_name):
    
    df_name.to_sql(name = table_name , con=engine, if_exists='append', index=False)

In [115]:
to_sql(data_2011, 'year_2011')
to_sql(data_2012, 'year_2012')
to_sql(data_2013, 'year_2013')
to_sql(data_2014, 'year_2014')
to_sql(data_2015, 'year_2015')
to_sql(data_2016, 'year_2016')
to_sql(data_2017, 'year_2017')

In [1]:
data_2012.head()

NameError: name 'data_2012' is not defined